In [25]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-05-06 20:58:22--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  22.9MB/s    in 1.1s    

2024-05-06 20:58:23 (22.9 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [26]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [123]:
df_users = df_ratings
counts2 = df_users['isbn'].value_counts()

df_users = df_users.groupby('user').filter(lambda x: len(x) >= 200)
df_users = df_users[~df_users['isbn'].isin(counts2[counts2 < 100].index)]
df_reduced = pd.merge(df_books, df_users, on='isbn')
df_dropped = df_reduced.drop_duplicates(subset=['user', 'isbn'])
df_dropped = df_reduced.drop_duplicates(subset=['user', 'title'])

df_pivot = df_dropped.pivot(index='title', columns='user', values='rating').fillna(0)


In [129]:
books_KNN = NearestNeighbors(metric='cosine', algorithm='auto')
books_KNN.fit(df_pivot)


NearestNeighbors(metric='cosine')

In [130]:
dist, inds = books_KNN.kneighbors(df_pivot.loc["Where the Heart Is (Oprah's Book Club (Paperback))"].values.reshape(1,-1))
print(dist)
print(list(reversed(df_pivot.index[inds[0]].tolist()))[:-1])

[[0.         0.7234864  0.7677075  0.7699411  0.77085835]]
['The Weight of Water', 'The Surgeon', 'I Know This Much Is True', 'The Lovely Bones: A Novel']


In [131]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  dist, inds = books_KNN.kneighbors(df_pivot.loc[book].values.reshape(1,-1))
  books = list(reversed(df_pivot.index[inds[0]].tolist()))[:-1]
  dist = list(reversed(dist[0]))[:-1]
  recs = [book, []]
  for i in range(4):
    recs[1].append([books[i], dist[i]])
  return recs


In [132]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
books

["Where the Heart Is (Oprah's Book Club (Paperback))",
 [['The Weight of Water', 0.77085835],
  ['The Surgeon', 0.7699411],
  ['I Know This Much Is True', 0.7677075],
  ['The Lovely Bones: A Novel', 0.7234864]]]

In [133]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
